In [2]:
import numpy as np
# Imports
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from surprise import Dataset
from surprise import Reader

from surprise import KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import SVD, SVDpp, NMF
from surprise import SlopeOne, CoClustering
%matplotlib inline
from blendingFunction import*

## Load Data

In [3]:
from data_related.data import Data

In [4]:
data = Data()

Preparing data ...
Splitting data to train and test data ...
... blend.
... data is splitted.
... data is prepared.


In [6]:
data.valid_df.head()

,user_id,movie_id,rating
16518,4275,10,3
443000,3725,370,5
933769,9639,742,4
456054,5624,389,4
1121681,7514,933,3


In [7]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(data.train_df, reader=reader)
test_data = Dataset.load_from_df(data.test_df, reader=reader)
valid_data = Dataset.load_from_df(data.valid_df, reader=reader)

In [75]:
trainset = train_data.build_full_trainset()

simoption = {'name' : 'pearson_baseline', 'user_based':True}
knnmeans_user = KNNWithMeans(k = 500, sim_options = simoption).fit(trainset)

test_data_np['ratingknnmeans_user'] = data.test_df[['user_id', 'movie_id']] \
.apply(lambda row: knnmeans_user.predict(row['user_id'], row['movie_id'])[3], axis=1)

valid_data_np['ratingknnmeans_user'] = valid_data_np[['user_id', 'movie_id']] \
.apply(lambda row: knnmeans_user.predict(row['user_id'], row['movie_id'])[3], axis=1)


simoption = {'name' : 'pearson_baseline', 'user_based':False}
knnmeans_item = KNNWithMeans(k = 57, sim_options = simoption).fit(trainset)


data.test_df['ratingknnmeans_item'] =data.test_df[['user_id', 'movie_id']] \
.apply(lambda row: knnmeans_item.predict(row['user_id'], row['movie_id'])[3], axis=1)

data.valid_df['ratingknnmeans_item'] = data.valid_df[['user_id', 'movie_id']] \
.apply(lambda row: knnmeans_item.predict(row['user_id'], row['movie_id'])[3], axis=1)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [63]:
valid_data_np.head()

,user_id,movie_id,rating,ratingknnmeans_user,ratingknnmeans_item
16518,4275,10,3,3.790300,3.643554
443000,3725,370,5,3.903508,4.056220
933769,9639,742,4,3.826401,4.081580
456054,5624,389,4,3.618853,3.553138
1121681,7514,933,3,3.748165,3.840001


In [84]:
from sklearn.preprocessing import MinMaxScaler


model_rating = ['ratingknnmeans_user','ratingknnmeans_item', 'rating']
model_blend = ['user_id', 'movie_id','ratingknnmeans_user','ratingknnmeans_item']
scaler = MinMaxScaler()

data.valid_df[model_rating] = scaler.fit_transform(valid_data_np[model_rating])
X_val = 

        user_id movie_id  rating  ratingknnmeans_user  ratingknnmeans_item
16518      4275       10    0.50             0.697575             0.660889
443000     3725      370    1.00             0.725877             0.764055
933769     9639      742    0.75             0.706600             0.770395
456054     5624      389    0.75             0.654713             0.638285
1121681    7514      933    0.50             0.687041             0.710000
421744     1152      350    1.00             0.660045             0.685614
201592     5602      176    0.75             0.786064             0.857255
793470     7733      643    1.00             0.822793             0.795347
799391     8183      645    1.00             0.859346             0.868871
295082     7378      247    0.50             0.736396             0.687399
184989     6277      161    1.00             0.839918             0.769126
196860     9395      172    0.25             0.586525             0.622975
391309     9140      320 

In [73]:
model = LogisticRegression()
model.fit(X_val,y_val)

/home/emma-hoggett/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)